In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

In [ ]:
# Config
DATASET_PATH = '../data/101_ObjectCategories'
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-3
IMG_SIZE = 224
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Transforms
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [ ]:
# Dataset (using ImageFolder structure)
full_dataset = datasets.ImageFolder(DATASET_PATH, transform=transform)

# Train/Test split
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
# Load pretrained ResNet18
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # freeze all layers

# Replace the final classification layer
# num_classes = len(full_dataset.dataset.classes)
num_classes = len(full_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [ ]:
model = model.to(DEVICE)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LR)

In [ ]:
# Training loop
def train_model(model, loader):
    model.train()
    for epoch in range(EPOCHS):
        running_loss = 0
        correct = 0
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

        acc = correct / len(loader.dataset) * 100
        print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {running_loss:.2f} | Train Acc: {acc:.2f}%")


In [ ]:
# Evaluation
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    acc = correct / len(loader.dataset) * 100
    print(f"\nTest Accuracy: {acc:.2f}%")


In [ ]:
# Run training and evaluation
train_model(model, train_loader)
evaluate_model(model, test_loader)

Epoch [1/10] Loss: 425.85 | Train Acc: 62.73%

Epoch [2/10] Loss: 130.60 | Train Acc: 88.80%

Epoch [3/10] Loss: 84.41 | Train Acc: 92.51%

Epoch [4/10] Loss: 64.48 | Train Acc: 93.85%

Epoch [5/10] Loss: 50.96 | Train Acc: 95.16%

Epoch [6/10] Loss: 41.79 | Train Acc: 96.12%

Epoch [7/10] Loss: 34.79 | Train Acc: 97.10%

Epoch [8/10] Loss: 30.16 | Train Acc: 97.40%

Epoch [9/10] Loss: 26.60 | Train Acc: 97.70%

Epoch [10/10] Loss: 24.10 | Train Acc: 97.79%


Test Accuracy: 90.16%